# Recommendation System

## I. Candidate Generation

### 1. Library Import

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import collections
import tensorflow as tf
from matplotlib import pyplot as plt
# !pip install tensorflow-recommenders
import tensorflow_recommenders as tfrs
from typing import Dict, Text
# !pip install surprise
# from surprise import SVD
# from surprise import Dataset
# from surprise import Reader

# from surprise import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 2. Exploratory Data Analysis

#### a. Import data

In [ ]:
movies_path = '/content/gdrive/My Drive/RS_dataset/movies_metadata.csv'
rating_path = '/content/gdrive/My Drive/RS_dataset/ratings_small.csv'

df_movies = pd.read_csv(movies_path)
df_rating = pd.read_csv(rating_path)

display(df_movies.head(5))
display(df_rating.head(5))

<ipython-input-3-f3b89c879428>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(movies_path)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
df_movies = df_movies.drop(columns='homepage')

In [ ]:
display(df_movies.columns)
display(df_movies.head(5))

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


#### Removing timestamp

In [ ]:
df_rating = df_rating.drop(columns=['timestamp'])
df_rating.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


In [ ]:
df_movie_dict = df_movies[['id', 'original_title']]
df_movie_dict.rename(columns={'id':'movieId'}, inplace=True)
display(df_movie_dict.head())
# print(df_movie_dict.to_numpy().unique().shape())

<ipython-input-14-466c7463eb85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_dict.rename(columns={'id':'movieId'}, inplace=True)


,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


#### b. another import

In [ ]:
ratings_df = pd.read_csv('/content/gdrive/My Drive/RS_dataset/ratings_small.csv')
df = pd.read_csv('/content/gdrive/My Drive/RS_dataset/movies_metadata.csv').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) # Incorrect data type
# ratings_df.drop('timestamp', axis=1)

df['id'] = df['id'].astype('int64')
# df.dropna(inplace=True)
df = df[~df['original_title'].duplicated()]
df = df.reset_index(drop=True)
# display(ratings_df['id'].describe())
# id_f = df['id'].unique()
# display(id_f.describe())
display(ratings_df['movieId'].describe())
ratings_df = ratings_df.merge(df[['id', 'original_title']], left_on='movieId',right_on='id', how='left')
# display(ratings_df['id'].describe())
ratings_df = ratings_df[~ratings_df['id'].isna()]
display(ratings_df['id'].describe())
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

ratings_df = ratings_df.drop(columns='timestamp')
display(ratings_df.head())

<ipython-input-3-04e519d4545f>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/gdrive/My Drive/RS_dataset/movies_metadata.csv').\


count    100004.000000
mean      12548.664363
std       26369.198969
min           1.000000
25%        1028.000000
50%        2406.500000
75%        5418.000000
max      163949.000000
Name: movieId, dtype: float64

count     43351.000000
mean       6011.507393
std       15943.886006
min           2.000000
25%         493.500000
50%        1387.000000
75%        2770.000000
max      160718.000000
Name: id, dtype: float64

,userId,movieId,rating,original_title
0,1,1371,2.5,Rocky III
1,1,1405,1.0,Greed
2,1,2105,4.0,American Pie
3,1,2193,2.0,My Tutor
4,1,2294,2.0,Jay and Silent Bob Strike Back


In [ ]:
movies_df = df[['id', 'original_title']]
movies_df.rename(columns={'id':'movieId'}, inplace=True)
movies_df.head()

<ipython-input-4-9ae786dff50d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df.rename(columns={'id':'movieId'}, inplace=True)


,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [ ]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

In [ ]:
# from sklearn.model_selection import train_test_split
train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

In [ ]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 43371
Unique users: 671


### 3. Model construction
#### Note:
- This session is under construction

#### b. Matrix Factorization Model class

##### activated

In [ ]:
device_name = tf.test.gpu_device_name()
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.12.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
model = CollabModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 146s 4s/step - root_mean_squared_error: 1.4533 - factorized_top_k/top_1_categorical_accuracy: 3.7143e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0076 - factorized_top_k/top_10_categorical_accuracy: 0.0195 - factorized_top_k/top_50_categorical_accuracy: 0.1067 - factorized_top_k/top_100_categorical_accuracy: 0.1754 - loss: 6814.3072 - regularization_loss: 0.0000e+00 - total_loss: 6814.3072
Epoch 2/3
35/35 [==============================] - 138s 4s/step - root_mean_squared_error: 1.0571 - factorized_top_k/top_1_categorical_accuracy: 6.8571e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0191 - factorized_top_k/top_10_categorical_accuracy: 0.0437 - factorized_top_k/top_50_categorical_accuracy: 0.2041 - factorized_top_k/top_100_categorical_accuracy: 0.3260 - loss: 6449.6936 - regularization_loss: 0.0000e+00 - total_loss: 6449.6936
Epoch 3/3
35/35 [==============================] - 135s 4s/step - root_mean_squared_error: 0.977

In [21]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 26s 3s/step - root_mean_squared_error: 1.0418 - factorized_top_k/top_1_categorical_accuracy: 9.7704e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0060 - factorized_top_k/top_10_categorical_accuracy: 0.0122 - factorized_top_k/top_50_categorical_accuracy: 0.0492 - factorized_top_k/top_100_categorical_accuracy: 0.0878 - loss: 5723.4220 - regularization_loss: 0.0000e+00 - total_loss: 5723.4220

Retrieval top-100 accuracy: 0.088
Ranking RMSE: 1.042


In [ ]:
save_path = 'MRS_v1.h5'
model.save_weights(save_path)

In [22]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

In [27]:
# Movie model index
index_M = tfrs.layers.factorized_top_k.BruteForce(model.movie_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

### Function for recommendation

In [30]:
# Recommendation function
# [@usrID]: num string user id
# [@showRating]: boolean to decide if showing rating.
def get_recommend(usrID, showRating):
  _, titles = index(tf.constant([usrID]))
  pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:5].numpy()]})
  display(pred_movies)


In [26]:
get_recommend('123', False)

,original_title
0,Un long dimanche de fiançailles
1,The Greatest Story Ever Told
2,Furankenshutain no Kaijū: Sanda tai Gaira
3,Astérix aux Jeux Olympiques
4,Pitch Black


In [32]:
# Recommendation function
# [@usrID]: num string user id
# [@showRating]: boolean to decide if showing rating.
def M_get_recommend(movie_titles, showRating):
  _, titles = index_M(tf.constant([movie_titles]))
  pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:5].numpy()]})
  display(pred_movies)

In [33]:
M_get_recommend('Scary Movie', False)

ValueError: ignored

#### a. Hyperparameter tunning

In [ ]:
embed_dim = 3 # embedding dimension, see https://www.nature.com/articles/s41467-021-23795-5 in detail


------------
### Using existed SVD model to predict rating for unseen movies
- Should be in the scoring section

In [ ]:
# using Surprise library
reader = Reader()
data = Dataset.load_from_df(df_rating, reader)

svd_model = SVD()
train_ds = data.build_full_trainset()
svd_model.fit(train_ds)


In [ ]:
svd_model.predict(1, 3671, 3)

-----------------

#### b. Train and store Matrix factorization model using selected hyperparameter

### 5. Model Evaluation

#### 1. Hyperparameter selection

### RMSE (Root Mean Square Error) and MAE (Mean Absolute Error)